### Notebook overview
- Goal: train a tiny, streaming pitch‑correction model that predicts per‑frame shift (cents) and a confidence.
- Strategy: build labels from a teacher (pYIN → low‑pass trend → hysteretic semitone snap), train a causal TCN on log‑mel features, export to int8 TFLite.
- Sections:
  1) Imports & config  
  2) Data windowing/normalization 
  3) Causal TCN
  4) Training (losses/callbacks)  
  5) Streaming emulator  
  6) SavedModel → TFLite


In [26]:
# Full Melody Pitch Correction (Causal TCN, Streaming)

# 1) Imports & Config
import os, json, glob, random
from pathlib import Path
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# directories for data access and artifacts
ROOT = Path("/Users/brandontsai/ESE3600/tinymlpitchcorrection")
PROC_DIR = ROOT / "data/processed/features"
META_DIR = ROOT / "metadata"
OUT_DIR = ROOT / "artifacts/full"
OUT_DIR.mkdir(parents=True, exist_ok=True)

with open(META_DIR / "feature_norm.json", "r") as f:
    norm = json.load(f)
FEATURE_MEAN = np.array(norm["feature_mean"] or [0.0]*64, dtype=np.float32)
FEATURE_STD = np.array(norm["feature_std"] or [1.0]*64, dtype=np.float32)
FPS = norm.get("frames_per_second", 100)

N_MELS = 64 # can decrease to 32 for faster training/smaller model
SHIFT_RANGE_CENTS = 300.0 # represents the range of possible pitch shifts

random.seed(13)
np.random.seed(13)
tf.random.set_seed(13)
print("Full melody config:", dict(n_mels=N_MELS, fps=FPS))


Full melody config: {'n_mels': 64, 'fps': 100}


In [27]:
# 2) Data loading (all categories), windowing, normalization

def list_npz(split):
    return sorted(glob.glob(str(PROC_DIR / split / '*.npz')))


def normalize_features(x):
    return (x - FEATURE_MEAN[None, :]) / (FEATURE_STD[None, :] + 1e-8)


# makes sequences of features and targets
def make_sequences(npz_paths, T=120, stride=60, limit=None):
    xs, ys = [], []
    count = 0
    for p in npz_paths:
        arr = np.load(p)
        logmel = normalize_features(arr['logmel'].astype(np.float32))
        target_shift = arr['target_shift'].astype(np.float32)
        L = len(target_shift)
        i = 0
        while i + T <= L:
            xs.append(logmel[i:i+T])
            ys.append(target_shift[i:i+T, None])
            i += stride
            count += 1
            if limit and count >= limit:
                break
        if limit and count >= limit:
            break
    return np.array(xs), np.array(ys)

train_files = list_npz('train')
val_files = list_npz('val')
X_train, y_train = make_sequences(train_files, T=120, stride=60)
X_val, y_val = make_sequences(val_files, T=120, stride=60)
print('Shapes:', X_train.shape, y_train.shape, X_val.shape, y_val.shape)

BATCH=32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(4096).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(tf.data.AUTOTUNE)


Shapes: (9463, 120, 64) (9463, 120, 1) (292, 120, 64) (292, 120, 1)


In [28]:
# 3) Causal TCN with dilations (causal Conv1D blocks)
from tensorflow.keras import layers as L, models

INPUT_DIM = N_MELS

inp = L.Input(shape=(None, INPUT_DIM), name='features')
x = L.Conv1D(64, 1, padding='same', activation='relu')(inp)

# Residual dilated blocks (replace depthwise conv with causal Conv1D to avoid padding='causal' limitation)
for d in [1, 2, 4]:
    res = x
    x = L.Conv1D(64, 3, dilation_rate=d, padding='same', activation=None)(x)
    x = L.Conv1D(64, 1, activation='relu')(x)
    x = L.Add()([x, res])

x = L.Conv1D(64, 1, activation='relu')(x)
shift = L.Conv1D(1, 1, activation=None, name='shift_cents')(x)
conf = L.Conv1D(1, 1, activation='sigmoid', name='confidence')(x)

model = models.Model(inputs=inp, outputs=[shift, conf])
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={'shift_cents':'mae','confidence':'binary_crossentropy'},
    loss_weights={'shift_cents':1.0,'confidence':0.01},
    metrics={'shift_cents':'mae'}
)
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, None, 64)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_32 (Conv1D)  │ (None, None, 64)  │      4,160 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_33 (Conv1D)  │ (None, None, 64)  │     12,352 │ conv1d_32[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_34 (Conv1D)  │ (None, None, 64)  │      4,160 │ conv1d_33[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, None, 64)  │          0 │ conv1d_34[0][0],  │
│                     │                   │            │ conv1d_32[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_35 (Conv1D)  │ (None, None, 64)  │     12,352 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_36 (Conv1D)  │ (None, None, 64)  │      4,160 │ conv1d_35[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, None, 64)  │          0 │ conv1d_36[0][0],  │
│                     │                   │            │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_37 (Conv1D)  │ (None, None, 64)  │     12,352 │ add_13[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_38 (Conv1D)  │ (None, None, 64)  │      4,160 │ conv1d_37[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, None, 64)  │          0 │ conv1d_38[0][0],  │
│                     │                   │            │ add_13[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_39 (Conv1D)  │ (None, None, 64)  │      4,160 │ add_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shift_cents         │ (None, None, 1)   │         65 │ conv1d_39[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ confidence (Conv1D) │ (None, None, 1)   │         65 │ conv1d_39[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,986 (226.51 KB)

 Trainable params: 57,986 (226.51 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# 4) Training with curriculum hook (chromatic -> scale-aware)

# packs the labels into a tuple of confidence and target shift
def pack_labels(y):
    conf = tf.ones_like(y, dtype=tf.float32)
    return (y, conf)

# reduces the learning rate on plateau and early stops if the validation loss does not improve
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_shift_cents_mae', mode='min', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_shift_cents_mae', mode='min', patience=7, restore_best_weights=True)
]

history = model.fit(
    train_ds.map(lambda x,y: (x, pack_labels(y))),
    validation_data=(X_val, pack_labels(y_val)),
    epochs=40,
    verbose=1,
    callbacks=callbacks
)

plt.figure(figsize=(6,3))
plt.plot(history.history['shift_cents_mae'], label='train MAE')
plt.plot(history.history['val_shift_cents_mae'], label='val MAE')
plt.legend(); plt.title('Cents MAE'); plt.show()


Epoch 1/40


KeyboardInterrupt: 

# Streaming Emulator 

 This section introduces the "Streaming Emulator", which simulates how the trained model would process audio input in a real-time, frame-by-frame (streaming) scenario. The emulator runs the model on one frame (feature vector) at a time, mimicking the constraints of live inference rather than batch processing. This is useful for testing how the model would behave in actual deployment on devices or applications that receive continuous audio input streams.


In [ ]:
# 5) Streaming emulator (per-hop inference)

class StreamingEmulator:
    def __init__(self, model, feature_mean, feature_std):
        self.model = model
        self.fm = feature_mean.astype(np.float32)
        self.fs = feature_std.astype(np.float32)
        self.buf = []

    def step(self, frame):
        frame = (frame - self.fm) / (self.fs + 1e-8)
        self.buf.append(frame)
        if len(self.buf) < 120:
            return 0.0, 0.0  

        if len(self.buf) > 120:
            self.buf.pop(0)

        x = np.array(self.buf, dtype=np.float32)[None, :, :] 
        shift, conf = self.model.predict(x, verbose=0)
        return float(shift[0,0,0]), float(conf[0,0,0])

# Example: run over first validation clip frame-by-frame
if len(val_files) > 0:
    arr = np.load(val_files[0])
    frames = arr['logmel']
    sim = StreamingEmulator(model, FEATURE_MEAN, FEATURE_STD)
    out_shift = []
    out_conf = []
    for t in range(frames.shape[0]):
        s,c = sim.step(frames[t])
        out_shift.append(s); out_conf.append(c)
    print('Streaming run frames:', len(out_shift))


AttributeError: 'StreamingEmulator' object has no attribute 'buf'

In [ ]:
# 6) Export to SavedModel and TFLite (int8)

export_dir = OUT_DIR / 'saved_model'
export_dir.mkdir(parents=True, exist_ok=True)

sig_inputs = tf.keras.Input(shape=(None, N_MELS), name='features', dtype=tf.float32)
sig_shift, sig_conf = model(sig_inputs)
serve = tf.keras.Model(inputs=sig_inputs, outputs={'shift_cents': sig_shift, 'confidence': sig_conf})

# Keras 3: export SavedModel for TFLite conversion
serve.export(str(export_dir))
print('SavedModel exported to', export_dir)

# Representative dataset windows

def rep_ds():
    for _ in range(256):
        i = np.random.randint(0, len(X_train))
        x = X_train[i:i+1].astype(np.float32)
        yield [x]

converter = tf.lite.TFLiteConverter.from_saved_model(str(export_dir))
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_ds
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()
(OUT_DIR / 'tflite').mkdir(parents=True, exist_ok=True)
with open(OUT_DIR / 'tflite' / 'full_melody.tflite', 'wb') as f:
    f.write(tflite_model)
print('TFLite model written.')


INFO:tensorflow:Assets written to: /Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model/assets


INFO:tensorflow:Assets written to: /Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model/assets


Saved artifact at '/Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None, 64), dtype=tf.float32, name='features')
Output Type:
  Dict[['shift_cents', TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None)], ['confidence', TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None)]]
Captures:
  4434867216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072243536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072243152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072240272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072240080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4690716688: TensorSpec(sha

W0000 00:00:1765332870.090141 19157689 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1765332870.090319 19157689 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-09 21:14:30.090487: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model
2025-12-09 21:14:30.091400: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-09 21:14:30.091406: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model
2025-12-09 21:14:30.100742: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-12-09 21:14:30.141820: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /Users/brandontsai/ESE3600/tinymlpitchcorrection/artifacts/full/saved_model
2025-12-09 21:14:30

TFLite model written.


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
2025-12-09 21:14:32.137970: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3705] Skipping runtime version metadata in the model. This will be generated by the exporter.


In [ ]:
tflite_output_name = OUT_DIR / 'tflite' / 'full_melody.tflite'
cc_output_name = OUT_DIR / 'cc' / 'full_melody.cc'
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {tflite_output_name} > {cc_output_name}
# Update variable names
REPLACE_TEXT = str(tflite_output_name).replace('/', '_').replace('.', '_')
!sed -i '' "s/{REPLACE_TEXT}/g_full_melody_model_data/g" {cc_output_name}

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite)
interpreter.allocate_tensors()

ops = interpreter.get_tensor_details()
print(interpreter._get_ops_details())


INFO:tensorflow:Assets written to: /var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_/assets


INFO:tensorflow:Assets written to: /var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_/assets


Saved artifact at '/var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None, 64), dtype=tf.float32, name='features')
Output Type:
  List[TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None)]
Captures:
  4434867216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072243536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072243152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072240272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072240080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13072242960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4690716688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  469070651

W0000 00:00:1765332873.378783 19157689 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1765332873.378798 19157689 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-09 21:14:33.378916: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_
2025-12-09 21:14:33.379688: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-09 21:14:33.379694: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_
2025-12-09 21:14:33.390243: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-12-09 21:14:33.432112: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/25/wsswbv_57nq7kwz4csy0cm500000gn/T/tmp30n3859_
2025-12-09 21:14:33.449766: I tensorflow/cc/saved_model/loader.c